In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedShuffleSplit


import sys
# sys.path.append("/Users/Work/Developer/interpretDL/interprettensor")
root_logdir = "./tf_logs"

# To plot pretty figures
%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
tf.__version__

Using TensorFlow backend.


'1.13.1'

In [2]:
# Helper Functions

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

######### Taken from sklearn #######
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots(figsize=[10,10])
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


def get1hot(y_train,y_test):
    from sklearn.preprocessing import OneHotEncoder

    enc = OneHotEncoder(categories="auto", sparse=False)
    y_train_1hot = enc.fit_transform([[label] for label in y_train]) # Since the function expects an array of "features" per sample
    y_test_1hot = enc.fit_transform([[label] for label in y_test])

    return y_train_1hot, y_test_1hot

def get_split(features, labels):
    
    # The train set will have equal amounts of each target class
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_index, test_index in split.split(features, labels):
        X_train = features[train_index]
        y_train = labels.iloc[train_index]
        X_test = features[test_index]
        y_test = labels.iloc[test_index]
        
        yield X_train, y_train, X_test, y_test

def plot_history(history):
    fig, axs = plt.subplots(1, 2, figsize=(18,6))
    
    # Plot training & validation accuracy values
    axs[0].grid(True)
    axs[0].plot(history.history['acc'])
    axs[0].plot(history.history['val_acc'])
    axs[0].set(title='Model accuracy', ylabel='Accuracy', xlabel='Epoch')
    axs[0].legend(['Train', 'Test'], loc='upper left')

    # Plot training & validation loss values
    axs[1].grid(True)
    axs[1].plot(history.history['loss'])
    axs[1].plot(history.history['val_loss'])
    axs[1].set(title='Model loss',ylabel='Loss', xlabel='Epoch')
    axs[1].legend(['Train', 'Test'], loc='upper left')
    
    plt.show()
    

def remove_label(features, labels, label="MCI"):
    labels = pd.Series(fused_labels)
    non_samples = labels != label

    stripped_features = features[non_samples]
    stripped_labels = labels[non_samples]

    return stripped_features, stripped_labels


In [3]:
# FLAGS

DROP_MCI = True # Whether to drop MCI samples or not

## Building a transformation pipeline
> **Recall that feature scaling only applies to training data**

In [20]:
from sklearn.base import BaseEstimator, TransformerMixin

class AttributeRemover(BaseEstimator, TransformerMixin):
    """
    Returns a copy of matrix with attributes removed
    """
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    
    def fit(self, X, y=None):
        return # Doesn't do anything
    
    def transform(self, X, y=None):
        return X.drop(columns=self.attribute_names)

class OverSampler(BaseEstimator, TransformerMixin):
    """
    Returns a copy of matrix with attributes removed
    """
    def __init__(self, random_state=42):
        self.smote = SMOTE(random_state=random_state)
    
    def fit(self, X, y=None):
        return None
    
    def transform(self, X, y=None):
        return self.smote.fit_resample(X,y)

### Getting data from csv

In [16]:
filename = "ICV_ADNI.csv"
raw_data = pd.read_csv(filename)
print(raw_data.info())
raw_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Columns: 300 entries, PTID to DX_bl
dtypes: float64(149), int64(148), object(3)
memory usage: 335.2+ KB
None


,PTID,scandate,ICV,G_and_S_frontomargin_SA_lh,G_and_S_frontomargin_TH_lh,G_and_S_occipital_inf_SA_lh,G_and_S_occipital_inf_TH_lh,G_and_S_paracentral_SA_lh,G_and_S_paracentral_TH_lh,G_and_S_subcentral_SA_lh,...,S_suborbital_TH_rh,S_subparietal_SA_rh,S_subparietal_TH_rh,S_temporal_inf_SA_rh,S_temporal_inf_TH_rh,S_temporal_sup_SA_rh,S_temporal_sup_TH_rh,S_temporal_transverse_SA_rh,S_temporal_transverse_TH_rh,DX_bl
0,094_S_2216,2011-05-04 08:35:04.461,307244.6,936,1.984,1158,2.107,993,2.306,1226,...,1.421,1007,1.937,763,1.759,4349,2.025,249,1.579,EMCI
1,029_S_2376,2011-07-05 18:17:58.518,303135.8,855,2.160,1291,2.287,1137,1.961,1451,...,2.457,1195,1.804,635,1.970,4895,2.071,344,1.642,EMCI
2,098_S_4003,2016-05-04 15:44:47.525,234729.1,849,2.122,909,2.272,788,2.430,717,...,2.222,626,1.870,784,1.826,3182,2.056,195,2.179,CN
3,021_S_2077,2014-10-21 15:26:50.834,278496.2,762,2.237,969,2.141,1153,2.130,1156,...,2.880,1127,2.245,789,1.924,4399,2.014,243,1.826,EMCI
4,021_S_5099,2013-06-11 14:47:47.885,221848.6,752,2.073,960,2.521,873,2.374,810,...,3.412,790,2.276,665,2.535,2914,2.168,183,2.166,EMCI


In [18]:
label_col = "DX_bl"
non_feature_cols = ["PTID", "scandate", "ICV",label_col]
# features = raw_data.drop(columns=["PTID", "scandate", "ICV",label_col])

features = AttributeRemover(attribute_names=non_feature_cols).transform(raw_data)

raw_labels = raw_data[label_col].copy()
ICVs = raw_data["ICV"].copy()
features.head()

,G_and_S_frontomargin_SA_lh,G_and_S_frontomargin_TH_lh,G_and_S_occipital_inf_SA_lh,G_and_S_occipital_inf_TH_lh,G_and_S_paracentral_SA_lh,G_and_S_paracentral_TH_lh,G_and_S_subcentral_SA_lh,G_and_S_subcentral_TH_lh,G_and_S_transv_frontopol_SA_lh,G_and_S_transv_frontopol_TH_lh,...,S_suborbital_SA_rh,S_suborbital_TH_rh,S_subparietal_SA_rh,S_subparietal_TH_rh,S_temporal_inf_SA_rh,S_temporal_inf_TH_rh,S_temporal_sup_SA_rh,S_temporal_sup_TH_rh,S_temporal_transverse_SA_rh,S_temporal_transverse_TH_rh
0,936,1.984,1158,2.107,993,2.306,1226,2.359,443,2.338,...,254,1.421,1007,1.937,763,1.759,4349,2.025,249,1.579
1,855,2.160,1291,2.287,1137,1.961,1451,2.059,685,2.207,...,309,2.457,1195,1.804,635,1.970,4895,2.071,344,1.642
2,849,2.122,909,2.272,788,2.430,717,2.606,474,2.456,...,250,2.222,626,1.870,784,1.826,3182,2.056,195,2.179
3,762,2.237,969,2.141,1153,2.130,1156,2.135,421,2.282,...,180,2.880,1127,2.245,789,1.924,4399,2.014,243,1.826
4,752,2.073,960,2.521,873,2.374,810,2.481,460,2.331,...,197,3.412,790,2.276,665,2.535,2914,2.168,183,2.166


In [19]:
# Getting all the columns related to surface area
thickness_features = [x for x in features.columns if "SA" in x ]

# Removing SA to reduce feature dimensions
raw_features = features.drop(columns=thickness_features)

raw_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Columns: 148 entries, G_and_S_frontomargin_TH_lh to S_temporal_transverse_TH_rh
dtypes: float64(148)
memory usage: 165.4 KB


> ### TODO: Compare LRP with and w/o ICV

In [141]:
# # Normalize by ICV
# features_icv_normed = raw_features.div(np.power(ICVs, 1/3), axis = "rows")
# features_icv_normed.head()

## Using SMOTE

> Generates interpolated samples to balance training data 

In [142]:
def balance_data(X,y):
    sm = SMOTE(random_state=42)
    
    features, labels = sm.fit_resample(X, y)
    
    print("Original: ", X.shape)
    print("After Data Augmentation: ", features.shape)
    
    return features, labels

## Fusing all the columns

In [143]:
# Mapping to convert labels
fuse_maps = {"SMC": "CN", "EMCI":"MCI", "LMCI":"MCI"}

# Lambda fucntion to be used with Map func
fuse = lambda x: fuse_maps[x] if x in fuse_maps else x
dist = lambda x: pd.Series(x).value_counts()/len(x)

fused_labels = pd.Series(list(map(fuse, raw_labels)))

print("Sample Size:", len(fused_labels))
print("Original:\n", dist(raw_labels))
print()
print("Fused:\n", dist(fused_labels))
pd.Series(raw_labels).value_counts()

Sample Size: 143
Original:
 EMCI    0.321678
CN      0.209790
AD      0.181818
LMCI    0.146853
SMC     0.139860
Name: DX_bl, dtype: float64

Fused:
 MCI    0.468531
CN     0.349650
AD     0.181818
dtype: float64


EMCI    46
CN      30
AD      26
LMCI    21
SMC     20
Name: DX_bl, dtype: int64

## Getting rid of MCI samples
> Only learning CN vs AD

In [144]:
features, labels = remove_label(raw_features, fused_labels) if DROP_MCI else (raw_features, fused_labels)
print("Sample Size:", len(labels))
# print(labels)
dist(labels)

Sample Size: 76


CN    0.657895
AD    0.342105
dtype: float64

In [145]:
# labels.reshape(-1,1)

### Get 1 Hot Vector representation of the *fused* categorical labels

In [146]:
# Converting labels to 1-Hot Vectors
from sklearn.preprocessing import OneHotEncoder

hot_encoder = OneHotEncoder(categories="auto", sparse=False)
hot_encoder.fit(labels.values.reshape(-1,1)) # Since the function expects an array of "features" per sample

print("Categories:", hot_encoder.categories_)
hot_encoder.transform(labels[:5].values.reshape(-1,1))


Categories: [array(['AD', 'CN'], dtype=object)]


array([[0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.]])

## Normalizing training inputs

Does not work at all without normalization. The ranges for surface area and thickness are vastly different.

In [147]:
from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler() #x-u/sd
# features = scaler.fit_transform(features) # Note that features is no longer a dataframe

NUM_FEATURES = features.shape[1]
NUM_LABELS = len(hot_encoder.categories_[0])

## Initial Split for Sanity Check

In [148]:
# Get split returns a generator
# List comprehension is one way to evaluate a generator
X_train, y_train, X_test, y_test = list(get_split(features, labels))[0]
print("Train Size:", X_train.shape)
print("Test Size:", y_test.shape)
# y_train

Train Size: (60, 148)
Test Size: (16,)


### Building the network

We will build a fully connected (slightly) deep network with no drop outs or batch normalization for now

In [149]:
from keras import optimizers
from keras import regularizers

def exp_decay(epoch):
    initial_lr = 0.01
    decay_steps = 50
    decay_rate = 0.1
    
    decayed_lr =  initial_lr * np.power(decay_rate, (epoch/decay_steps))
#     print("New Learning Rate:", decayed_lr)
    return decayed_lr

def build_dnn(num_features, num_labels=3):
    keras.backend.clear_session()
    reset_graph()
    
    reg_scale = 0.001 # For L1 Reg
    my_reg = regularizers.l1_l2(reg_scale) # Can change this if needed
    
    dnn = keras.models.Sequential()

    Dense = keras.layers.Dense

    # Using He initialization
    he_init = keras.initializers.he_normal()
    

    dnn.add(Dense(units = 150, activation="elu", input_dim=num_features,
                  kernel_initializer=he_init, kernel_regularizer = my_reg))
    dnn.add(keras.layers.Dropout(0.5))
    dnn.add(Dense(units = 100, activation="elu",
                  kernel_initializer=he_init, kernel_regularizer = my_reg))
    dnn.add(keras.layers.Dropout(0.5))
    dnn.add(Dense(units=50, activation='elu',
                  kernel_initializer=he_init, kernel_regularizer = my_reg))
    dnn.add(keras.layers.Dropout(0.5))
    
    dnn.add(Dense(units=num_labels, activation="softmax",
                  kernel_initializer=he_init, kernel_regularizer = my_reg)) # 5 labels -> logits for now
    
    nadam = keras.optimizers.Nadam()
    NSGD = keras.optimizers.SGD(lr=exp_decay(0),momentum=0.9,nesterov=True)
    
    dnn.compile(loss='categorical_crossentropy',
                  optimizer=NSGD,
                  metrics=['accuracy'])
    
    return dnn

dnn = build_dnn(NUM_FEATURES, NUM_LABELS)
dnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 150)               22350     
_________________________________________________________________
dropout_1 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               15100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 102       
Total para

In [150]:
def train_model(model, X, y, X_test=[], y_test=[], epochs=30, batch_size=20, verbose=1, plot=True):
    
    X_train,y_train = balance_data(X,y) # Both are np arrays now
    
    X_train = scaler.fit_transform(X_train)
    
    y_train = hot_encoder.transform(y_train.reshape(-1,1))
    y_test = hot_encoder.transform(y_test.values.reshape(-1,1))
    
    lr_scheduler = keras.callbacks.LearningRateScheduler(exp_decay)
    
    callback_list = []
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size = batch_size, validation_data=(X_test, y_test),
                       callbacks=callback_list, verbose=verbose)
    
    if plot: plot_history(history)
    
    return history
    

In [151]:
dnn = build_dnn(NUM_FEATURES, NUM_LABELS)
history = train_model(dnn, X_train, y_train, X_test, y_test, epochs=20, batch_size=10)

Original:  (60, 148)
After Data Augmentation:  (78, 148)
Train on 78 samples, validate on 16 samples
Epoch 1/20
78/78 [==============================] - 1s 7ms/step - loss: 11.6805 - acc: 0.5897 - val_loss: 10.7933 - val_acc: 0.6875
Epoch 2/20
78/78 [==============================] - 0s 184us/step - loss: 11.2255 - acc: 0.7051 - val_loss: 11.0492 - val_acc: 0.8125
Epoch 3/20
78/78 [==============================] - 0s 174us/step - loss: 10.9869 - acc: 0.7692 - val_loss: 10.8099 - val_acc: 0.8750
Epoch 4/20
78/78 [==============================] - 0s 164us/step - loss: 10.8288 - acc: 0.8205 - val_loss: 10.5460 - val_acc: 0.8750
Epoch 5/20
78/78 [==============================] - 0s 191us/step - loss: 9.9648 - acc: 0.8462 - val_loss: 11.1995 - val_acc: 0.8750
Epoch 6/20
78/78 [==============================] - 0s 169us/step - loss: 9.5050 - acc: 0.9231 - val_loss: 11.0076 - val_acc: 0.8750
Epoch 7/20
78/78 [==============================] - 0s 193us/step - loss: 9.2554 - acc: 0.9231 - va

FigureCanvasNbAgg()

> AD vs CN

loss: 3.4295 - acc: 0.8846 - val_loss: 3.9303 - val_acc: 0.8125 - w/ ICV
val_acc = 0.875 -w/o ICV

### Calculating a confusion matrix

In [152]:
def make_confusion(model, X_test, y_test):
    y_pred_probs = dnn.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(hot_encoder.transform(y_test.values.reshape(-1,1)), axis=1)
    plot_confusion_matrix(y_true, y_pred, classes=hot_encoder.categories_[0])
make_confusion(dnn, X_test, y_test)

Confusion matrix, without normalization
[[5 0]
 [2 9]]


FigureCanvasNbAgg()

In [153]:
hot_encoder.categories_[0]

array(['AD', 'CN'], dtype=object)

## Using K=10 Fold Cross Validation

In [204]:
from sklearn.model_selection import StratifiedKFold as KFold


def getKF(X,y, n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42 ) #Default = 10

    for train_index, test_index in kf.split(X,y):
        X_train = X[train_index]
        y_train = y.iloc[train_index]
        X_test = X[test_index]
        y_test = y.iloc[test_index]
        
        yield X_train, y_train, X_test, y_test, test_index

histories = []
testing_indxs =[]
predictions = []
true_labels = []
zoo = []
for X_train, y_train, X_test, y_test, test_index in getKF(features, labels):
    print(test_index)
    dnn = build_dnn(NUM_FEATURES, NUM_LABELS)
    history = train_model(dnn,X_train, y_train, X_test, y_test, verbose=0, plot=False, epochs=100, batch_size=10)
    
    # Updating all information arrays
    histories.append(history)
    testing_indxs.append(test_index)
    zoo.append(dnn)
    
    y_pred_probs = dnn.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(hot_encoder.transform(y_test.values.reshape(-1,1)), axis=1)
    
    predictions.extend(y_pred)
    true_labels.extend(y_true)
    
    print("Scores on test set: loss={:0.3f} accuracy={:.4f}".format(history.history["acc"][-1], history.history["val_acc"][-1]))

[ 2 19 23 34 42 49 64 71]
Original:  (68, 148)
After Data Augmentation:  (90, 148)
Scores on test set: loss=1.000 accuracy=0.7500
[25 33 35 36 40 47 63 74]
Original:  (68, 148)
After Data Augmentation:  (90, 148)
Scores on test set: loss=1.000 accuracy=0.7500
[ 3  5  6 10 17 45 58 61]
Original:  (68, 148)
After Data Augmentation:  (90, 148)
Scores on test set: loss=1.000 accuracy=0.7500
[ 8 13 18 21 43 67 72 73]
Original:  (68, 148)
After Data Augmentation:  (90, 148)
Scores on test set: loss=1.000 accuracy=0.8750
[11 14 15 22 32 44 48 57]
Original:  (68, 148)
After Data Augmentation:  (90, 148)
Scores on test set: loss=1.000 accuracy=0.7500
[ 0  7 38 50 54 55 56 70]
Original:  (68, 148)
After Data Augmentation:  (90, 148)
Scores on test set: loss=1.000 accuracy=0.7500
[ 1 16 28 41 51 60 66]
Original:  (69, 148)
After Data Augmentation:  (90, 148)
Scores on test set: loss=0.989 accuracy=0.8571
[ 4 29 31 53 59 65 69]
Original:  (69, 148)
After Data Augmentation:  (90, 148)
Scores on tes

In [207]:
plot_confusion_matrix(predictions, true_labels, classes=hot_encoder.categories_[0])

Confusion matrix, without normalization
[[18  7]
 [ 8 43]]


FigureCanvasNbAgg()

In [208]:
plot_confusion_matrix(predictions, true_labels, classes=hot_encoder.categories_[0], normalize = True)

Normalized confusion matrix
[[0.72       0.28      ]
 [0.15686275 0.84313725]]


FigureCanvasNbAgg()

In [158]:
fig, axs = plt.subplots(len(histories)//2,2, figsize=(20,20))
axs=axs.flatten()
dfs = []

for i,history in enumerate(histories):
    df = pd.DataFrame(history.history)
    dfs.append(df)
    df[["acc","val_acc"]].plot(ax=axs[i])

FigureCanvasNbAgg()

In [40]:
# val_accs = [df["val_acc"].iloc[-1] for df in dfs]
# print("Average:",np.mean(val_accs))
# plt.bar(x=range(10),height=val_accs)
# # plt.scatter(x=range(10), y=np.mean(val_accs))
# plt.xlabel("Fold Number")
# plt.ylabel("Acc")
# plt.title("Accuracies for Each Fold")
# val_accs

## Lets pick the best model from the folds...

In [209]:
idx = -2
# from keras import backend as K
# K.clear_session()
best_dnn = zoo[idx]
# validation = labels.iloc[testing_indxs[-3]]
# best_dnn.save("best_dnn.h5")
best_dnn.evaluate(features,hot_encoder.transform(labels.values.reshape(-1,1)))

76/76 [==============================] - 0s 59us/step


[0.3019657919281407, 0.9736842136634024]

### Evaluating on non SMOTE Samples

In [210]:
# def get_correct_predictions(model,X,y):

model = best_dnn

samples = features

# hot_enc = OneHotEncoder(categories="auto", sparse=False)
sample_labels = hot_encoder.transform([[label] for label in labels])
print("Categories:", hot_encoder.categories_)

predictions = model.predict(samples)
preds = np.array([np.argmax(x) for x in predictions])
true_labels = np.array([np.argmax(x) for x in sample_labels])

correct = preds == true_labels
AD_Sample = true_labels == 0

correct_preds = preds[correct]
correct_preds.shape

Categories: [array(['AD', 'CN'], dtype=object)]


(74,)

In [211]:
print("SANITY CHECK")
loss_and_metrics = model.evaluate(samples[correct], sample_labels[correct])
print("Scores on test set: loss={:0.3f} accuracy={:.4f}".format(*loss_and_metrics))

SANITY CHECK
74/74 [==============================] - 0s 52us/step
Scores on test set: loss=0.281 accuracy=1.0000


# LRP

In [184]:
import innvestigate
import innvestigate.utils as iutils


def perform_analysis(model, analyzer, data, labels):
    analysis = analyzer.analyze(data)
    prediction = model.predict(data)
    
    df_anal = pd.DataFrame(analysis)
    
    return df_anal

# Selecting a DNN
test_dnn = dnn

# Stripping the softmax activation from the model
model_wo_sm = iutils.keras.graph.model_wo_softmax(test_dnn)

# Creating an analyzer
# lrp = innvestigate.create_analyzer("lrp.z", model_wo_sm)
lrp = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPAlpha2Beta1(model=model_wo_sm)

# Getting correctly predicted samples with AD
test_idx = correct & AD_Sample    
test_samples = samples[test_idx] 
test_labels = sample_labels[test_idx]

AD_lrp = perform_analysis(dnn,lrp,test_samples, test_labels)

# test_idx = correct & (~AD_Sample)    
# test_samples = samples[test_idx] 
# test_labels = sample_labels[test_idx]

# # perform_analysis(nn,gradient_analyzer,flowers,types)
# CN_lrp = perform_analysis(dnn,lrp,test_samples, test_labels)

# Getting all the samples that can be correctly predicted
test_idx = correct
all_samples = samples[test_idx] 
all_labels = sample_labels[test_idx]

# perform_analysis(nn,gradient_analyzer,flowers,types)
all_lrp = perform_analysis(dnn,lrp, all_samples, all_labels)

In [164]:
# print(AD_lrp.shape)
# print(CN_lrp.shape)
print(AD_lrp.shape)
# ALL_lrp

(26, 148)


In [185]:
lrp_results = all_lrp
# population = lrp_results.mean()
# population.plot()
# plt.show()

In [198]:
# Average for entire population
plt.figure()
all_lrp.mean().plot()

FigureCanvasNbAgg()

In [103]:
sorted_features = population.sort_values(ascending=False)
best_features = sorted_features[:6]

In [186]:
best_features.index

Int64Index([96, 94, 46, 22, 110, 134], dtype='int64')

In [187]:
# abs(lrp_results)[best_features.index].hist(figsize=(20,20))

In [188]:
raw_features.columns[best_features.index]

Index(['G_oc-temp_med-Parahip_TH_rh', 'G_oc-temp_lat-fusifor_TH_rh',
       'S_circular_insula_ant_TH_lh', 'G_oc-temp_med-Parahip_TH_lh',
       'G_temporal_inf_TH_rh', 'S_oc-temp_med_and_Lingual_TH_rh'],
      dtype='object')

In [189]:
plt.figure()
sorted_features.plot(kind="bar", figsize=[20,10])
plt.xticks(rotation=65, fontsize="small")

FigureCanvasNbAgg()

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147]), <a list of 148 Text xticklabel objects>)

In [216]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pos_only = lrp_results.copy()
pos_only[pos_only < 0] = 0
pca.fit(pos_only)

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [217]:
print("Variance: ", pca.explained_variance_ratio_)
# pca.components_

Variance:  [0.38462503 0.10845578]


In [218]:
X = pca.transform(pos_only)
df = pd.DataFrame(X, columns=["PC1","PC2"])
_labels = np.array([np.argmax(x) for x in all_labels])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 2 columns):
PC1    75 non-null float64
PC2    75 non-null float64
dtypes: float64(2)
memory usage: 1.2 KB


In [247]:
# plt.figure(figsize=(20,10))
df.plot.scatter(x="PC1", y="PC2", s= 30, c=_labels, colormap='winter',figsize=(12,8))
# plt.legend(["AD", "CN"])
print("Categories:", hot_encoder.categories_)

FigureCanvasNbAgg()

Categories: [array(['AD', 'CN'], dtype=object)]


# Plot PCA for 3 components

In [220]:
from mpl_toolkits.mplot3d import Axes3D

In [221]:
pca3 = PCA(n_components=3)
pca3.fit(pos_only)
print("Variance: ", pca3.explained_variance_ratio_)

pc_3d = pca3.transform(pos_only)

Variance:  [0.38462503 0.10845578 0.09197229]


In [227]:
%matplotlib widget
fig = plt.figure(figsize=(12,10))
ax = fig.add_subplot(111, projection='3d')

FigureCanvasNbAgg()

In [228]:
ax.scatter(pc_3d[:,0], pc_3d[:,1], pc_3d[:,2], c=_labels, s=40)
ax.set(xlabel="PC1", ylabel="PC2", zlabel="PC3")
plt.show()

## Try tSNE

In [239]:
from sklearn.manifold import TSNE
# pca_reduced = PCA(n_components=50)
tSNE = TSNE(n_components=2, init="pca", random_state=42)
tSNE_relevance = tSNE.fit_transform(pos_only)

In [242]:
X = tSNE_relevance
fig = plt.figure(figsize=(12,10))
tSNE_relevance.shape

FigureCanvasNbAgg()

(75, 2)

In [246]:
plt.scatter(x=X[:,0], y=X[:,1], s= 30, c=_labels)